In [ ]:
import geopandas as gpd
import h3pandas
import pandas as pd

# Data sourced from Tailte Eireann Surveying Open Data Portal

gdf = gpd.read_file("Local_Electoral_Areas_-_National_Statutory_Boundaries_-_2019.geojson")


In [ ]:
# Slice down to RPZs, taken from https://www.housingagency.ie/rent-pressure-zones
# As of Nov 2023 the following Local Election Areas are designated RPZs:
# All of the counties Dublin (4 LAs), Kildare, Louth, Meath, Wicklow, Kilkenny, Limerick, Waterford, Westmeath.
# Specific LEAs: Carlow (Carlow). Graiguecullen/Portarlington, Portlaoise (Laoise). Tullamore (Offaly). Gorey (Wexford). Ennis, Shannon (Clare),
#          Cork City, Ballincollig/Carrigaline, Bandon/Kinsale, Cobh, Fermoy, Macroom, Mallow, Midleton (Cork). Killarney (Kerry).
#          Galway City, Athenry/Oranmore, Gort/Kinvara (Galway). Westport (Mayo). Sligo/Strandhill (Sligo).

# Not elegant but otherwise need to create an empty gdf with the right cols and geometry
wgdf = gdf.loc[gdf['COUNTY'].eq('DUBLIN')]

counties = ['KILDARE','LOUTH','MEATH','WICKLOW','KILKENNY','LIMERICK','WATERFORD','WESTMEATH']

for county in counties:
    wgdf = pd.concat([wgdf, gdf.loc[gdf['COUNTY'].eq(county)]])

# Athlone is here, because it's in both Westmeath and is the only RPZ in Roscommon. Including Roscommon isn't clear from the Housing Agency, 
# but their map shows both. This'll cause a duplicate.
leas = ['ATHLONE','CARLOW','GRAIGUECULLEN','PORTLAOISE','TULLAMORE','GOREY','ENNIS','SHANNON','CORK CITY','CARRIGALINE','BANDON',\
       'COBH','FERMOY','MACROOM','MALLOW','MIDLETON','KILLARNEY','GALWAY CITY','ATHENRY-ORANMORE','GORT-KINVARA','WESTPORT','SLIGO-STRANDHILL']

for lea in leas:
    wgdf = pd.concat([wgdf, gdf.loc[gdf['ENGLISH'].str.startswith(lea)]])    

# Drop the duplicate Athlone (Westmeath)
wgdf.drop_duplicates(inplace = True)

# Drop the LEA in Limerick that was removed on the map but not described in text
wgdf.drop(gdf[gdf['ENGLISH'].str.contains('CAPPAMORE')].index, inplace=True)

# Drop Ennistimon and Enniscorthy, which came in with Ennis*
wgdf.drop(gdf[gdf['ENGLISH'].str.contains('ENNISTIMON')].index, inplace=True)
wgdf.drop(gdf[gdf['ENGLISH'].str.contains('ENNISCORTHY')].index, inplace=True)


wgdf.to_file("rpz-Nov2023.geojson", driver='GeoJSON')

In [ ]:
# Resample to H3 cells if you want
resolution = 8
h3gdf = wgdf.h3.polyfill_resample(resolution)
h3gdf.to_file('RPZH3.geojson', driver='GeoJSON')